# University Towns Housing Prices Hypothesis Testing

In [86]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Hypothesis Testing


Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.



In [87]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [154]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    import pandas as pd
    import re
    with open('university_towns.txt', "r") as file:
        lines = file.readlines()
    new_lines = []
    for line in lines:
        if not re.match(r'^\s*$', line):
            new_lines.append(line)

    lines = new_lines.copy()
    for index, line in enumerate(lines):
        lines[index] = line.strip()
    df = pd.DataFrame(columns=('State', 'RegionName'))
    i = 0  
    state_string = ""  
    region_string = ""  
    for line in lines:
        if '[edit]' in line:
            state_string = line.replace('[edit]', "")
        else:
            region_string = re.sub(r' \(.*', "", line)
            df.loc[i] = [state_string, region_string]
            i += 1
            
    return df


#get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [99]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    #pd.set_option('display.max_rows', None)
    x = pd.read_excel('gdplev.xls', skiprows=7, usecols=[4,6])
    x.columns = ['quarter', 'gdp']
    x['year'] = [int(x[:4]) for x in x['quarter']]
    x = x[x['year']>=2000]
    x['recession']= x['gdp'].diff()
    x = x.reset_index()
    for i in range(1, len(x)-1):
        if (x.loc[i, 'recession'] <0) & (x.loc[i+1, 'recession'] <0):
            x.loc[i, 'recession'] = True
        start_index = (x.loc[x['recession']==True].index).min()
    
    return x.loc[start_index, 'quarter']
   
#get_recession_start()

'2008q3'

In [100]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    x = pd.read_excel('gdplev.xls', skiprows=7, usecols=[4,6])
    x.columns = ['quarter', 'gdp']
    x['year'] = [int(x[:4]) for x in x['quarter']]
    x = x[x['year']>=2000]
    x['recession']= x['gdp'].diff()
    x = x.reset_index()
    for i in range(3, len(x)):
        if (x.loc[i-3, 'recession']<0) & (x.loc[i-2, 'recession']<0) & (x.loc[i-1, 'recession']>0) & (x.loc[i, 'recession']>0):
            x.loc[i,'recession'] = True
        end_index = (x.loc[x['recession']==True].index).min()
       
    return x.loc[end_index, 'quarter']

#get_recession_end()

'2009q4'

In [101]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    x = pd.read_excel('gdplev.xls', skiprows=7, usecols=[4,6])
    x.columns = ['quarter', 'gdp']
    x['year'] = [int(x[:4]) for x in x['quarter']]
    x = x[x['year']>=2000]
    x['recession']= x['gdp'].diff()
    x = x.reset_index()
    x['recession_start']= False
    x['recession_end']= False
    for i in range(1, len(x)-1):
            if (x.loc[i, 'recession'] <0) & (x.loc[i+1, 'recession'] <0):
                x.loc[i, 'recession_start'] = True
            recession_start = (x.loc[x['recession_start']==True].index).min()
    for i in range(3, len(x)):
            if (x.loc[i-3, 'recession']<0) & (x.loc[i-2, 'recession']<0) & (x.loc[i-1, 'recession']>0) & (x.loc[i, 'recession']>0):
                x.loc[i,'recession_end'] = True
            recession_end = (x.loc[x['recession_end']==True].index).min()
    result=x.loc[recession_start: recession_end]
    
    return (result.loc[result['gdp']==(result['gdp']).min(),'quarter']).values[0]

#get_recession_bottom()

'2009q2'

In [106]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    import pandas as pd
    x= pd.read_csv('City_Zhvi_AllHomes.csv')
    for item in x.columns:
        if item[-2:] in ['01','02','03']:
            x.rename(columns={item: item[:4] +'q1'}, inplace=True)
        if item[-2:] in ['04','05','06']:
            x.rename(columns={item: item[:4] +'q2'}, inplace=True)
        if item[-2:] in ['07','08','09']:
            x.rename(columns={item: item[:4] +'q3'}, inplace=True)
        if item[-2:] in ['10','11','12']:
            x.rename(columns={item: item[:4] +'q4'}, inplace=True)

    columns_to_keep = ['RegionName', 'State','2000q1', '2000q1',
           '2000q1', '2000q2', '2000q2', '2000q2', '2000q3', '2000q3',
           '2000q3', '2000q4', '2000q4', '2000q4', '2001q1', '2001q1',
           '2001q1', '2001q2', '2001q2', '2001q2', '2001q3', '2001q3',
           '2001q3', '2001q4', '2001q4', '2001q4', '2002q1', '2002q1',
           '2002q1', '2002q2', '2002q2', '2002q2', '2002q3', '2002q3',
           '2002q3', '2002q4', '2002q4', '2002q4', '2003q1', '2003q1',
           '2003q1', '2003q2', '2003q2', '2003q2', '2003q3', '2003q3',
           '2003q3', '2003q4', '2003q4', '2003q4', '2004q1', '2004q1',
           '2004q1', '2004q2', '2004q2', '2004q2', '2004q3', '2004q3',
           '2004q3', '2004q4', '2004q4', '2004q4', '2005q1', '2005q1',
           '2005q1', '2005q2', '2005q2', '2005q2', '2005q3', '2005q3',
           '2005q3', '2005q4', '2005q4', '2005q4', '2006q1', '2006q1',
           '2006q1', '2006q2', '2006q2', '2006q2', '2006q3', '2006q3',
           '2006q3', '2006q4', '2006q4', '2006q4', '2007q1', '2007q1',
           '2007q1', '2007q2', '2007q2', '2007q2', '2007q3', '2007q3',
           '2007q3', '2007q4', '2007q4', '2007q4', '2008q1', '2008q1',
           '2008q1', '2008q2', '2008q2', '2008q2', '2008q3', '2008q3',
           '2008q3', '2008q4', '2008q4', '2008q4', '2009q1', '2009q1',
           '2009q1', '2009q2', '2009q2', '2009q2', '2009q3', '2009q3',
           '2009q3', '2009q4', '2009q4', '2009q4', '2010q1', '2010q1',
           '2010q1', '2010q2', '2010q2', '2010q2', '2010q3', '2010q3',
           '2010q3', '2010q4', '2010q4', '2010q4', '2011q1', '2011q1',
           '2011q1', '2011q2', '2011q2', '2011q2', '2011q3', '2011q3',
           '2011q3', '2011q4', '2011q4', '2011q4', '2012q1', '2012q1',
           '2012q1', '2012q2', '2012q2', '2012q2', '2012q3', '2012q3',
           '2012q3', '2012q4', '2012q4', '2012q4', '2013q1', '2013q1',
           '2013q1', '2013q2', '2013q2', '2013q2', '2013q3', '2013q3',
           '2013q3', '2013q4', '2013q4', '2013q4', '2014q1', '2014q1',
           '2014q1', '2014q2', '2014q2', '2014q2', '2014q3', '2014q3',
           '2014q3', '2014q4', '2014q4', '2014q4', '2015q1', '2015q1',
           '2015q1', '2015q2', '2015q2', '2015q2', '2015q3', '2015q3',
           '2015q3', '2015q4', '2015q4', '2015q4', '2016q1', '2016q1',
           '2016q1', '2016q2', '2016q2', '2016q2', '2016q3', '2016q3']
    x = x[columns_to_keep] 
    x['State'] = x['State'].map(states)
    x= x.set_index(['State','RegionName'])

    return x.groupby(x.columns, axis=1).mean()

#convert_housing_data_to_quarters()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10730 entries, (New York, New York) to (Wisconsin, Holland)
Data columns (total 67 columns):
2000q1    9355 non-null float64
2000q2    9357 non-null float64
2000q3    9374 non-null float64
2000q4    9376 non-null float64
2001q1    9409 non-null float64
2001q2    9410 non-null float64
2001q3    9420 non-null float64
2001q4    9420 non-null float64
2002q1    9435 non-null float64
2002q2    9436 non-null float64
2002q3    9454 non-null float64
2002q4    9455 non-null float64
2003q1    9509 non-null float64
2003q2    9509 non-null float64
2003q3    9568 non-null float64
2003q4    9568 non-null float64
2004q1    9623 non-null float64
2004q2    9625 non-null float64
2004q3    9666 non-null float64
2004q4    9673 non-null float64
2005q1    9730 non-null float64
2005q2    9739 non-null float64
2005q3    9772 non-null float64
2005q4    9778 non-null float64
2006q1    9807 non-null float64
2006q2    9807 non-null float64
2006q3    9818 non-null f

In [162]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''

    uni_towns = get_list_of_university_towns()
    uni_towns = uni_towns[['State', 'RegionName']]
    uni_towns['uni_town'] = True
    uni_towns = uni_towns.set_index(['State', 'RegionName'])
    data = convert_housing_data_to_quarters()
    data = data.merge(uni_towns, how = 'outer', left_index = True, right_index = True)
    data.uni_town[data.uni_town.isnull()] = False

    start = get_recession_start()
    bottom = get_recession_bottom()
    df = data[[start, bottom, 'uni_town']]
    df['ratio'] = df['2009q2'] - df['2008q3']
    
    ut = df[df['uni_town'] == 1]['ratio'].dropna()
    not_ut = df[df['uni_town'] == 0]['ratio'].dropna()
    t = ttest_ind(ut, not_ut)
    p = t[1]
    
    different = False
    if p < 0.01:
        different = True
    
    better = 'university town'
    if ut.mean() < not_ut.mean():
        better = 'non-university town'
        
    return (different, p, better)


#run_ttest()

(True, 0.0043252148535112009, 'university town')